In [11]:
#######################################################
#### Set this value to the directory of period-suite ##
#######################################################
pathToSuite="/Users/sertoez/git-projects/period-suite/"
#######################################################

import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()
# The following may be overridden in input file, but need not be specified there
#
#
defaultPrecision=1e-100;
# Do not forget to increase the precision of initial conditions if going beyond e-100
#
#
#
precision=[]; #if empty we will use the default precision for all systems
classicalPeriods=true; #unless overridden 
paths=[]; #unless overridden we will convert this to a list of straight paths

t0=time.time();
load(pathToSuite+"current.sage")
t1=time.time();
print "Loaded file in", t1-t0, "seconds";
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "\nIntegrating system number", j;
    transitionMatricies = []
    for i in [1..noOfdeqs]:
        tt0=time.time()
        tm=deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true)
        print "\tODE number", i, "took", time.time()-tt0, "seconds to integrate"
        transitionMatricies=transitionMatricies+[tm]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    print "The largest error:"
    print max([max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies])
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(2000), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:noOfdeqs]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print periods



# Write the periods to file

def row2string(row):
    strRow=str(row[0])
    for i in [1..(len(row)-1)]:
        strRow=strRow+", "+str(row[i])
    return "["+strRow+"]"
    
outputFile = open(pathToSuite+"lastPeriods",'w')

outputFile.write("M:=Matrix(["+row2string(periods[0]))
for i in [1..(periods.nrows()-1)]:
    outputFile.write(",\n"+row2string(periods[i]))
outputFile.write("]);")
outputFile.close()


Loaded file in 0.918778181076 seconds
Integrating systems 1 through 3

Integrating system number 1
	ODE number 1 took 0.0539638996124 seconds to integrate
	ODE number 2 took 0.0526978969574 seconds to integrate
	ODE number 3 took 0.135092973709 seconds to integrate
	ODE number 4 took 0.123594999313 seconds to integrate
	ODE number 5 took 0.122972011566 seconds to integrate
	ODE number 6 took 0.228930950165 seconds to integrate
	ODE number 7 took 0.107169866562 seconds to integrate
	ODE number 8 took 0.0228171348572 seconds to integrate
	ODE number 9 took 0.0233380794525 seconds to integrate
	ODE number 10 took 0.087583065033 seconds to integrate
	ODE number 11 took 0.0258979797363 seconds to integrate
	ODE number 12 took 0.068351984024 seconds to integrate
	ODE number 13 took 0.0618789196014 seconds to integrate
	ODE number 14 took 0.0228469371796 seconds to integrate
	ODE number 15 took 0.0789170265198 seconds to integrate
	ODE number 16 took 0.161885023117 seconds to integrate
	ODE n

In [7]:
classicalPeriods

True